# Chapter 06 - Decision Trees

## Training and Visualizing a Decision Tree

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [4]:
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

In [5]:
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [24]:
import os

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, "images", fig_id)

In [25]:
from sklearn.tree import export_graphviz

export_graphviz(tree_clf, out_file=image_path("ch06_iris_tree.dot"),
                feature_names=iris.feature_names[2:], class_names=iris.target_names,
                rounded=True, filled=True)

## Making Predictions

In [37]:
!dot -Tpng ch06_iris_tree.dot -o ch06_iris_tree.png 

Error: dot: can't open ch06_iris_tree.dot


The decision tree classification process is shown below:

![decision_tree](./images/iris_tree.png)

- the *samples* attribute counts how many training instances it applies to. For example, 100 training instances have a petal length greater than 2.45 cm (depth 1, right), and of those 100, 54 have a petal width smaller than 1.75 cm (depth 2, left).
- the *value* attribute tells you how many training instances of each class this node applies to: for example, the bottom-right node applies to 0 Iris setosa, 1 Iris versicolor, and 45 Iris virginica.
- Finally, a node’s *gini* attribute measures its impurity: a node is “pure” (gini=0) if all training instances it applies to belong to the same class. For example, since the depth-1 left node applies only to Iris setosa training instances, it is pure and its gini score is 0.

Equation 6.1 show how the training algorithm computes the *gini* score $G_i$ of the $i^{th}$ node:

$$G_i = 1 - \sum_{k=1}^n p_{i, k}^2$$

- $p_{i, k}$ is the ratio of class $k$ instances among the training instances in the $i^{th}$ node.

*The depth-2 left node has a gini score equal to* $1 - (0/54)^2 - (49/54)^2 - (5/54)^2$.

## Estimating Class Probabilities

In [44]:
# Features used to train the model
iris["feature_names"][2:]

['petal length (cm)', 'petal width (cm)']

In [47]:
print(X[0])
tree_clf.predict_proba([X[0]])

[1.4 0.2]


array([[1., 0., 0.]])

## The CART training algorithm

Scikit-Learn uses the *Classification and Regression Tree* (CART) algorithm to train Decision Trees (also called “growing” trees). The algorithm works by first splitting the training set into two subsets using a single feature k and a threshold $t_k$ (e.g., “petal length ≤ 2.45 cm”). How does it choose k and $t_k$? It searches for the pair (k,  $t_k$) that produces the purest subsets (weighted by their size). Equation 6-2 gives the cost function that the algorithm tries to minimize.

$$
J(k, t_k) = \frac{m_{left}}{m} G_{left} + \frac{m_{right}}{m} G_{right}
$$

where:

- $G_{left/right}$ measures the impurity of the left/right subset
- $m_{left/right}$ is the number of instance in the left/right subset



Once the CART algorithm has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets, and so on, recursively. It stops recursing once it reaches the maximum depth (defined by the max_depth hyperparameter), or if it cannot find a split that will reduce impurity. 

## Computational Complexity

Making predictions requires traversing the Decision Tree from the root to a leaf. Decision Trees generally are approximately balanced, so traversing the Decision Tree requires going through roughly O(log (m)) nodes. Since each node only requires checking the value of one feature, the overall prediction complexity is O(log (m)), independent of the number of features. So predictions are very fast, even when dealing with large training sets.

The training algorithm compares all features (or less if max_features is set) on all samples at each node. Comparing all features on all samples at each node results in a training complexity of O(n × m log (m)). For small training sets (less than a few thousand instances), Scikit-Learn can speed up training by presorting the data (set presort=True), but doing that slows down training considerably for larger training sets.

## Gini Impurity or Entropy?

By default, the Gini impurity measure is used, but you can select the entropy impurity measure instead by setting the *criterion* hyperparameter to "entropy".

In Machine Learning, entropy is frequently used as an impurity measure: a set’s entropy is zero when it contains instances of only one class. Equation 6-3 shows the definition of the entropy of the $i^{th}$ node

$$H_i = - \sum_{k=1}^n p_{i, k}\log_2(p_{i, k})$$

So, should you use Gini impurity or entropy? The truth is, most of the time it does not make a big difference: they lead to similar trees. Gini impurity is slightly faster to compute, so it is a good default. However, when they differ, Gini impurity tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.

## Regularization Hyperparameters

The DecisionTreeClassifier class has a few other parameters that similarly restrict the shape of the Decision Tree: *min_samples_split* (the minimum number of samples a node must have before it can be split), *min_samples_leaf* (the minimum number of samples a leaf node must have), *min_weight_fraction_leaf* (same as *min_samples_leaf* but expressed as a fraction of the total number of weighted instances), *max_leaf_nodes* (the maximum number of leaf nodes), and *max_features* (the maximum number of features that are evaluated for splitting at each node). Increasing min_* hyperparameters or reducing max_* hyperparameters will regularize the model.

## Regression

The CART algorithm works mostly the same way as earlier, except that instead of trying to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the MSE. Equation 6-4 shows the cost function that the algorithm tries to minimize.

$$
J(k, t_k) = \frac{m_{left}}{m} MSE_{left} + \frac{m_{right}}{m} MSE_{right}
$$


where

- $MSE_{node} = \sum_{i \in node} (\hat{y}_{node} - y^{(i)})^2$
- $\hat{y}_{node} = \frac{1}{m_{node}} \sum_{i \in node} y^{(i)}$


## Instability

Hopefully by now you are convinced that Decision Trees have a lot going for
them: they are simple to understand and interpret, easy to use, versatile, and
powerful. However, they do have a few limitations. First, as you may have
noticed, Decision Trees love orthogonal decision boundaries (all splits are
perpendicular to an axis), which makes them sensitive to **training set rotation**.

More generally, the main issue with Decision Trees is that **they are very sensitive
to small variations in the training data**. Actually, since the training algorithm used by Scikit-Learn is stochastic, you may get very different models even on the same training data (unless you set the *random_state* hyperparameter).

Random Forests can limit this instability by averaging predictions over many trees, as we will see in the next chapter.